# sklearnでp値を計算する方法

## ライブラリのインポート

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn import linear_model

## データの読み込み

In [2]:
# データを読み込みます
data = pd.read_csv('1.02. Multiple linear regression.csv')

# はじめの5つのデータを確認します
data.head()

,SAT,"Rand 1,2,3",GPA
0,1714,1,2.40
1,1664,3,2.52
2,1760,3,2.54
3,1685,3,2.74
4,1693,2,2.83


In [3]:
# 統計量を表示します
data.describe()

,SAT,"Rand 1,2,3",GPA
count,84.000000,84.000000,84.000000
mean,1845.273810,2.059524,3.330238
std,104.530661,0.855192,0.271617
min,1634.000000,1.000000,2.400000
25%,1772.000000,1.000000,3.190000
50%,1846.000000,2.000000,3.380000
75%,1934.000000,3.000000,3.502500
max,2050.000000,3.000000,3.810000


## 重回帰分析モデルを作成します

### 従属変数と独立変数を定義します

In [4]:
# There are two independent variables: 'SAT' and 'Rand 1,2,3'
x = data[['SAT','Rand 1,2,3']]

# and a single depended variable: 'GPA'
y = data['GPA']

## p値をsklearnを使って計算する方法

In [5]:
import scipy.stats as stat

class LinearRegression(linear_model.LinearRegression):
    """
    LinearRegression class after sklearn's, but calculate t-statistics
    and p-values for model coefficients (betas).
    Additional attributes available after .fit()
    are `t` and `p` which are of the shape (y.shape[1], X.shape[1])
    which is (n_features, n_coefs)
    This class sets the intercept to 0 by default, since usually we include it
    in X.
    """
    
    # nothing changes in __init__
    def __init__(self, fit_intercept=True, normalize=False, copy_X=True,
                 n_jobs=1):
        self.fit_intercept = fit_intercept
        self.normalize = normalize
        self.copy_X = copy_X
        self.n_jobs = n_jobs
        self.positive = False

    
    def fit(self, X, y, n_jobs=1):
        self = super(LinearRegression, self).fit(X, y, n_jobs)
        
        # SSEを計算します (誤差の二乗の合計)
        # 標準誤差を計算します
        sse = np.sum((self.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
        se = np.array([np.sqrt(np.diagonal(sse * np.linalg.inv(np.dot(X.T, X))))])

        # t値を求めます
        self.t = self.coef_ / se
        # p値を求めます
        self.p = np.squeeze(2 * (1 - stat.t.cdf(np.abs(self.t), y.shape[0] - X.shape[1])))
        return self

In [6]:
# クラスからオブジェクトを作成します
reg_with_pvalues = LinearRegression()
reg_with_pvalues.fit(x,y)

LinearRegression()

In [7]:
# オブジェクトの中の属性pを表示します
reg_with_pvalues.p

array([0.        , 0.75717067])

In [8]:
# 新しいデータフレームを作成します
reg_summary = pd.DataFrame([['SAT'],['Rand 1,2,3']],columns =['Features'])
# 係数の列を追加します
reg_summary['Coefficients'] = reg_with_pvalues.coef_
# p値を列に追加します
reg_summary['p-values'] = reg_with_pvalues.p.round(3)

In [9]:
# 結果の表を作成します
# --> P値が0.5より大きい「Rand 1,2,3」は、有用じゃない、といえる。
# --> 変数として使えない、ということ。
reg_summary

,Features,Coefficients,p-values
0,SAT,0.001654,0.000
1,"Rand 1,2,3",-0.008270,0.757
